In [1]:
from stackapi import StackAPI
import os
import json
import time
import pandas as pd
import numpy as np

# 2 - Preprocess meanings

In [2]:
path = '../Dati wiki/pulito_nodes.csv'

In [3]:
df = pd.read_csv(path, error_bad_lines=False)

In [4]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

In [5]:
tags = df['Tag']
meaning = df['Significati']

In [6]:
def clean_text(corpus):
    clear_text = []
    for phrase in corpus:
        # Replace URL with the tag url
        phrase = re.sub("https*\S+", "url", phrase)
        # Remove strange characters
        # &quot => ""
        # &#39 => '
        # &amp; => &
        # &lt; => <
        # &gt; => >
        phrase = re.sub("&quot;|\r|\n\|&#39|&amp;|&lt;|&gt;", "", phrase)

        # Punctuaction removal
        phrase = re.sub('[%s]' % re.escape(string.punctuation), ' ', phrase)
        # Replace numbers with the tag number
        phrase = re.sub("\d+", "number ", phrase)
        # Convert to lowercase
        phrase = phrase.lower()
        # Replace the over spaces
        phrase = re.sub('\s{2,}', " ", phrase)
        clear_text.append([phrase])
    return clear_text

In [7]:
cleared_text = clean_text(meaning)

In [8]:
import networkx as nx
import itertools
from operator import itemgetter

In [9]:
G = nx.read_graphml("../DataSet FFF/Graph_data/Real_Network.graphml")

In [10]:
degree_dict = dict(G.degree(G.nodes()))
nx.set_node_attributes(G, degree_dict, 'degree')
sorted_degree = sorted(degree_dict.items(), key = itemgetter(1), reverse = True)

In [11]:
sample_nodes_597 = []
for i in sorted_degree:
    if i[1] > 50:
        sample_nodes_597.append(i[0])
print(len(sample_nodes_597))

257


In [12]:
def select_sample_meaning(sample, meaning):
    sampled_meaning = []
    sample_nodes = []
    for i in sample:
        for j in range(len(tags)):
            if tags[j]==i:
                sampled_meaning.append(i + " " + meaning[j][0])
                sample_nodes.append(i)
    return sampled_meaning, sample_nodes

In [13]:
sampled_meaning_597, sample_nodes_597 = select_sample_meaning(sample_nodes_597, cleared_text)

In [14]:
"""
for i in range(len(sample_nodes_1057)):
    if not isNaN(tags[i]):
        sampled_meaning_1057[i] = sample_nodes_1057[i] + " " + sampled_meaning_1057[i][0]
    else:
        sampled_meaning_1057[i] = "nan " + sampled_meaning_1057[i][0]
"""

'\nfor i in range(len(sample_nodes_1057)):\n    if not isNaN(tags[i]):\n        sampled_meaning_1057[i] = sample_nodes_1057[i] + " " + sampled_meaning_1057[i][0]\n    else:\n        sampled_meaning_1057[i] = "nan " + sampled_meaning_1057[i][0]\n'

In [15]:
word_tokens = []
for phrase in sampled_meaning_597:
    word_tokens.append(phrase.split())
stop_words = stopwords.words('english')
no_sw_597=[]
for phrase in word_tokens:
    no_sw_tmp=[]
    for word in phrase:
        if word not in stop_words:
            no_sw_tmp.append(word)
    no_sw_597.append(no_sw_tmp)

In [16]:
cleared_meaning_597 = []
for i in no_sw_597:
    new_s = ""
    for j in i:
        new_s = new_s + " " + j
    cleared_meaning_597.append(new_s)

In [17]:
df = pd.DataFrame({"Tags": sample_nodes_597, "Meaning" : cleared_meaning_597})
df.to_csv("nodes_597_meaning.csv", index=False)

In [18]:
from gensim.models import Word2Vec

In [19]:
word2vec_597 = Word2Vec(no_sw_597, min_count=1)

In [20]:
len(word2vec_597.wv)

2444

In [21]:
def check_isin(word2vec, data):
    not_in = []
    for i in data:
        try:
            word2vec.wv[i]
        except:
            not_in.append(i)
    print(len(not_in))

# 3 - Store word2vec embedding

In [22]:
from linkpred.evaluation import Pair
import pickle

In [23]:
H_597 = G.subgraph(sample_nodes_597)

In [24]:
first = []
second = []
for i in H_597.edges():
    first.append(word2vec_597.wv[i[0]])
    second.append(word2vec_597.wv[i[1]])
new_df = pd.DataFrame({"first":first, "second":second})
new_df.reset_index(inplace=True, drop=True)
new_df.to_pickle("embedding_pickle_597.csv")

In [25]:
word2vec_597.save("word2vec_597.model")

In [26]:
nodes = list(H_597.nodes())
universe_597 = set([Pair(i) for i in itertools.product(nodes, nodes) if i[0]!=i[1]])

In [27]:
with open('universe_597.pickle', 'wb') as f:
    pickle.dump(universe_597, f)

In [28]:
first = []
second = []
for i in H_597.edges():
    first.append(i[0])
    second.append(i[1])

In [29]:
new_df = pd.DataFrame({"first":first, "second":second})
new_df.reset_index(inplace=True, drop=True)
new_df.to_pickle("edges_original_597.csv")

In [30]:
nx.write_graphml(H_597,"H_597.graphml")